# **Extract data through Google API**

In [35]:
import json
from typing import List, Dict, Any, Tuple
import pandas as pd
import geopandas as gpd
import requests
import datetime
import matplotlib.pyplot as plt

##### **Steps**:
To get the data from the Google Bucket into workable format, we follow:
* Connect with the API
* **Expand this step!** Set-up pipeline which downloads all relevant data per country code
* Format the data into DateTime (y) and features (x)
* Store externally as .csv

##### **Background info -- API**
The API has three basic functions:
* *ListGauges*: Returns a list of supported gauges within a given country (code)
* *GetGaugeModel*: Returns model information of a specific gauge (within a given country, that is)
* *QueryGaugeForecasts*: Returns the forecasts of specific gauge

In [36]:
def get_API_key(path : str) -> str:
    """
    Get the API key from a .txt file stored elsewhere

    :param path: path to the .txt file containing the API key
    :return: the API key
    """
    with open(path, 'r') as f:
        return f.read().strip()

# Note: add this folder manually at the pointed path for the script to run
# For the key, ask the owner of the repository or try brute force :)
path_API_key = '../../data/keys/API_key.txt'

In [37]:
def get_country_codes(path : str) -> Dict:
    """
    Get the country codes from a .json file stored elsewhere

    :param path: path to the .json file containing the country codes
    :return: the country codes
    """
    with open(path, 'r') as f:
        return json.load(f)


path_country_codes = '../../data/country_codes.json'
dict_country_codes = get_country_codes(path_country_codes)

In [38]:
def generate_url_ListGauges(country_code : str, path_to_key : str) -> str:
    """
    Generate the URL to get the list of gauges for a specific country

    :param country_code: the country code
    :param path_to_key: path to the .txt file containing the API key
    :return: the URL
    """

    # URL from the Colab
    base_url = 'https://floodforecasting.googleapis.com/v1/gauges:searchGaugesByArea'
    return f'{base_url}?key={get_API_key(path_to_key)}'

    # base_url = 'https://floodforecasting.googleapis.com/v1/gauges'
    # return f'{base_url}?filter=region_code={country_code}&key={get_API_key(path_to_key)}'


def get_ListGauges(country_code : str, path_to_key : str) -> List[Dict[str, Any]]:
    """
    Get the list of gauges for a specific country

    :param url: the URL
    :return: the list of gauges
    """
    url = generate_url_ListGauges(country_code, path_to_key)
    response = requests.post(url, json = {'regionCode': country_code})

    if response.status_code != 200:
        raise Exception(f'Error: {response.status_code} -- {response.text}')
    
    try:
        return response.json()['gauges']
    except ValueError as exc:
        raise Exception(f'Error parsing .json: {exc} -- {response.text}')

In [39]:
print(generate_url_ListGauges('ML', path_API_key))

https://floodforecasting.googleapis.com/v1/gauges:searchGaugesByArea?key=AIzaSyAGeDCPtAeSdZ4V_Yyo1ya2S66YlOqbqI


In [40]:
list_gauges = get_ListGauges('ML', path_API_key)

Exception: Error: 400 -- {
  "error": {
    "code": 400,
    "message": "API key not valid. Please pass a valid API key.",
    "status": "INVALID_ARGUMENT",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "API_KEY_INVALID",
        "domain": "googleapis.com",
        "metadata": {
          "service": "floodforecasting.googleapis.com"
        }
      }
    ]
  }
}
